# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient # I'm importing my labrary
client = MongoClient("localhost:27017") # I'm giving my local adress
db = client["Ironhack"] # name of the database
c = db.get_collection("companies") # name of the collection

In [68]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [ ]:
filter = {"name":"Babelgum"}
projection = { "_id":0, "name": 1,}
results = list(c.find(filter, projection))
results

[{'name': 'Babelgum'}]

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [15]:
filter = {"name":"Babelgum"}
projection = { "_id":0, "name": 1,}
results = list(c.find(filter, projection))
results

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [19]:
filter = {"number_of_employees":5000}
projection = { "_id":0, "name": 1,}
results = list(c.find(filter, projection).sort("number_of_employees", -1).limit(20))
results

[{'name': 'Spotify'},
 {'name': 'PCH International'},
 {'name': 'McAfee'},
 {'name': 'JDS Uniphase Corporation'},
 {'name': 'Air Astana Airlines'},
 {'name': 'United Internet'},
 {'name': 'Vcare Call Center'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [70]:
filter = {"number_of_employees":{'$gte': 2000, '$lt': 2005}}
projection = { "_id":0, "name": 1,"founded_year": 1,}
results = list(c.find(filter, projection).sort("number_of_employees", -1))
results

[{'name': 'Netflix', 'founded_year': 1997},
 {'name': 'Razorfish', 'founded_year': 1995},
 {'name': 'Aspect Software', 'founded_year': 1973},
 {'name': 'WJT Global Solutions', 'founded_year': 2008},
 {'name': 'CEGID', 'founded_year': 1983},
 {'name': 'AppLabs', 'founded_year': 2001},
 {'name': 'Aztecsoft', 'founded_year': 1995},
 {'name': 'DiGi Telecommunications', 'founded_year': 1995},
 {'name': 'Emerson Network Power', 'founded_year': None}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [73]:
filter = {"ipo.valuation_amount":{'$gt': 100000000},"founded_year": {'$lt': 2010}}
projection = { "_id":0, "name": 1,"ipo": 1}
results = list(c.find(filter, projection))
results

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}},
 {'name': 'Brightcove',
  

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [79]:
filter = {"number_of_employees":{'$lt': 1000},"founded_year": {'$lt': 2005}}
projection = { "_id":0, "name": 1}
results = list(c.find(filter, projection).sort("number_of_employees").limit(10))
results

[{'name': 'Skype'},
 {'name': 'Simpy'},
 {'name': 'Eurekster'},
 {'name': 'Ticketmaster'},
 {'name': 'Fox Interactive Media'},
 {'name': 'stylediary'},
 {'name': 'Monster'},
 {'name': 'EditGrid'},
 {'name': 'Compete'},
 {'name': 'MindTouch'}]

### 6. All the companies that don't include the `partners` field.

In [109]:
filter = {"partners": []}
projection = {"_id":0, "name": 1}
results = list(c.find(filter, projection))
results

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'},
 {'name': 'Geni'},
 {'name': 'Flektor'},
 {'name': 'Fox Interactive Media'},
 {'name': 'Twitter'},
 {'name': 'StumbleUpon'},
 {'name': 'Gizmoz'},
 {'name': 'Scribd'},
 {'name': 'Slacker'},
 {'name': 'Lala'},
 {'name': 'Helio'},
 {'name': 'eBay'},
 {'name': 'MeetMoi'},
 {'name': 'Joost'},
 {'name': 'CBS'},
 {'name': 'Viacom'},
 {'name': 'Babelgum'},
 {'name': 'Plaxo'},
 {'name': 'Cisco'},
 {'name': 'Yahoo!'},
 {'name': 'Powerset'},
 {'name': 'Technorati'},
 {'name': 'SpinVox'},
 {'name': 'AddThis'},
 {'name': 'OpenX'},
 {'name': 'Mahalo'},
 {'name': 'Sparter'},
 {'name': 'Kyte'},
 {'name': 'Veoh'},
 {'name': 'Gannett'},
 {'name': 'Thoof'},
 {'name': 'Jingle Networks'},
 {'name': 'Info'},
 {'name': 'JotSpot'},
 {'name': 'Meetup'},
 {'name': 'Mercora'},
 {'name': 'NetRatings'},
 {'name': 'LifeLock'},
 {'name': 'Wesabe'},
 {'name': 'Jangl

### 7. All the companies that have a null type of value on the `category_code` field.

In [85]:
filter = {"category_code": {"$type": 10}} 
projection = { "_id":0, "name": 1}
results = list(c.find(filter, projection))
results

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'},
 {'name': 'Repeater Store'},
 {'name': 'Tapesh'},
 {'name': 'iPersians'},
 {'name': 'NewPersia'},
 {'name': 'Pyra Labs'},
 {'name': 'Feedmap'},
 {'name': 'NuvoMedia'},
 {'name': 'Intwine'},
 {'name': 'The Weinstein Company'},
 {'name': 'ExecuNet'},
 {'name': 'A1Vacations'},
 {'name': 'TripHomes'},
 {'name': 'Picturehouse'},
 {'name': 'Covalent Technologies'},
 {'name': 'Epic Trip'},
 {'name': 'Telepost'},
 {'name': 'Fidelity'},
 {'name': 'Quirk'},
 {'name': 'Skyrove'},
 {'name': 'Leechblock'},
 {'name': 'My Success Gateway'},
 {'name': 'RealNames'},
 {'name': 'The Easynet Group'},
 {'name': 'NetNames'},
 {'name': 'Sniperoo'},
 {'name': 'JD Project'},
 {'name': 'ABC News'},
 {'name': 'Teoma Technologies'},
 {'name': 'cometro'},
 {'name': 'U-Cast'},
 {'na

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [89]:
filter = {"number_of_employees":{'$gte': 100, '$lt': 1000}}
projection = { "_id":0, "name": 1, "number_of_employees":1}
results = list(c.find(filter, projection))
results

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [90]:
projection = { "_id":0, "name": 1}
results = list(c.find(filter, projection).sort("ipo",-1))
results

[{'name': 'GREE'},
 {'name': 'Zillow'},
 {'name': 'Yelp'},
 {'name': 'Higher One'},
 {'name': 'OpenTable'},
 {'name': 'QlikTech'},
 {'name': 'Millennial Media'},
 {'name': 'Mixi'},
 {'name': 'Cornerstone OnDemand'},
 {'name': 'Infoblox'},
 {'name': 'SolarWinds'},
 {'name': 'Wix'},
 {'name': 'Marketo'},
 {'name': 'Shutterfly'},
 {'name': 'Rally Software'},
 {'name': 'Brightcove'},
 {'name': 'ChannelAdvisor'},
 {'name': 'DemandTec'},
 {'name': 'KIT digital'},
 {'name': 'QuinStreet'},
 {'name': 'Geeknet'},
 {'name': 'Criteo'},
 {'name': 'Veeva'},
 {'name': 'FireEye'},
 {'name': 'Cvent'},
 {'name': 'Silver Spring Networks'},
 {'name': 'Workday'},
 {'name': 'LifeLock'},
 {'name': 'WANdisco'},
 {'name': 'KAYAK'},
 {'name': 'Splunk'},
 {'name': 'Bazaarvoice'},
 {'name': 'Zynga'},
 {'name': 'Procera Networks'},
 {'name': 'Procera Networks'},
 {'name': 'FriendFinder Networks'},
 {'name': 'Epocrates'},
 {'name': 'Tesla Motors'},
 {'name': 'DigitalGlobe'},
 {'name': 'Infinera Corporation'},
 {'na

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [93]:
projection = { "_id":0, "name": 1, "number_of_employees":1}
results = list(c.find(filter, projection).sort("number_of_employees",-1).limit(10))
results

[{'name': 'Datamonitor', 'number_of_employees': 984},
 {'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Relax Solutions Pvt Ltd', 'number_of_employees': 900},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'InMobi', 'number_of_employees': 900}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [94]:
filter = {"founded_month": {'$gt':10}}
projection = { "_id":0, "name": 1}
results = list(c.find(filter, projection).limit(1000))
results

[{'name': 'Omnidrive'},
 {'name': 'Plaxo'},
 {'name': 'Kyte'},
 {'name': 'Thoof'},
 {'name': 'Wesabe'},
 {'name': 'SmugMug'},
 {'name': 'Ikan'},
 {'name': 'blinkx'},
 {'name': 'Zlio'},
 {'name': 'Fleck'},
 {'name': 'SideStep'},
 {'name': 'RockYou'},
 {'name': 'Swivel'},
 {'name': 'Sway'},
 {'name': 'FlickIM'},
 {'name': 'Gumiyo'},
 {'name': 'Piczo'},
 {'name': 'Socialtext'},
 {'name': 'hi5'},
 {'name': 'Clipperz'},
 {'name': 'Indeed'},
 {'name': 'Grockit'},
 {'name': 'Adobe Systems'},
 {'name': 'Phonezoo'},
 {'name': 'Wishpot'},
 {'name': 'IntenseDebate'},
 {'name': 'TextPayMe'},
 {'name': 'PayPal'},
 {'name': 'Mokitown'},
 {'name': 'Neopets'},
 {'name': 'MindTouch'},
 {'name': 'ChaCha'},
 {'name': 'MobiTV'},
 {'name': 'BitPass'},
 {'name': 'TubeMogul'},
 {'name': 'dpreview'},
 {'name': 'Multiply'},
 {'name': 'Tutor'},
 {'name': 'imeem'},
 {'name': 'AdMob'},
 {'name': 'PeekYou'},
 {'name': 'WooMe'},
 {'name': 'Orgoo'},
 {'name': 'YourStreet'},
 {'name': 'FlowPlay'},
 {'name': 'ThisNext

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [101]:
filter = {"founded_year": {'$lt': 2000}, "acquisition.price_amount": {'$gt': 10000000}}
projection = { "_id":0, "name": 1}
results = list(c.find(filter, projection))
results

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'},
 {'name': 'Neopets'},
 {'name': 'Sun Microsystems'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Sabre'},
 {'name': 'Lastminute'},
 {'name': 'Sportsline'},
 {'name': 'Digitas'},
 {'name': 'Webshots'},
 {'name': 'MIVA'},
 {'name': 'iVillage'},
 {'name': 'PriceGrabber'},
 {'name': 'LinkShare'},
 {'name': 'Kaboose'},
 {'name': 'Fastclick'},
 {'name': 'Eloqua'},
 {'name': 'Omniture'},
 {'name': 'Interwoven'},
 {'name': 'HowStuffWorks'},
 {'name': 'RealAge'},
 {'name': 'Match'},
 {'name': 'Xdrive'},
 {'name': 'Alexa'},
 {'name': 'Offermatica'},
 {'name': 'ioko'},
 {'name': 'MyNewPlace'},
 {'name': 'DoubleClick'},
 {'name': 'Danger'},
 {'name': 'Direct Hit'},
 {'name': 'BEA Systems'},
 {'name': 'Napster'},
 {'name': 'SafeNet'},
 {'name': 'LowerMyBills'},
 {'name': 'Various'},
 {'name': 'aQuantive'},
 {'name': 'PeopleSupport'},
 {'name': 'Wedding Channel'},
 {'name': 'iC

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [103]:
filter = {"acquisition.acquired_year": {'$gt': 2010}}
projection = { "_id":0, "name": 1, "acquisition":1}
results = list(c.find(filter, projection).sort("acquisition.price_amount"))
results

[{'name': 'Geni',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2012/11/28/all-in-the-family-myheritage-buys-former-yammer-stablemate-geni-com-raises-25m/',
   'source_description': 'MyHeritage acquires Geni and $25M to build family tree of the whole world',
   'acquired_year': 2012,
   'acquired_month': 11,
   'acquired_day': 28,
   'acquiring_company': {'name': 'MyHeritage', 'permalink': 'myheritage'}}},
 {'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'blogT

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [108]:
filter = {"founded_year": {'$ne': None}} # I ahve removed the companies with field = as NONE, because that was making the infor dirty.
projection = { "_id":0, "name": 1, "founded_year":1}
results = list(c.find(filter, projection).sort("founded_year"))
results

[{'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'US Army', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'McKesson', 'founded_year': 1833},
 {'name': 'Bachmann Industries', 'founded_year': 1833},
 {'name': 'Bertelsmann', 'founded_year': 1835},
 {'name': 'Accuity', 'founded_year': 1836},
 {'name': 'CENTRA', 'founded_year': 1839},
 {'name': 'WeGame', 'founded_year': 1840},
 {'name': 'VideoSurf', 'founded_year': 1840},
 {'name': 'VideoSurf', 'founded_year': 1840},
 {'name': 'The Economist Group', 'founded_year': 1843},
 {'name': 'Pearson', 'founded_year': 1844},
 {'name': 'Associated Press', 'founded_year': 1846},
 {'name': 'Beloit College', 'founded_year': 1846},
 {'name': 'Siemens', 'founded_year': 1847},
 {'name': 'Stiefel', 'founded_year': 1847},
 {'name': 'Pfizer', 'founded_year': 1848},
 {'name': 'Nykredit Realkredit', 'founded_year': 1851},
 {'name': 'Corning', 'founded_year': 1851},
 {'name': 'L

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [129]:
filters = {"founded_day": {"$lte": 7}}
projection = {'name': 1, "_id": 0}
results = list(c.find(filters, projection).sort("acquisition.price_amount", 1).limit(10))
results

[{'name': 'eBay'},
 {'name': 'Joost'},
 {'name': 'Fox Interactive Media'},
 {'name': 'Geni'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Technorati'},
 {'name': 'Yahoo!'},
 {'name': 'Babelgum'},
 {'name': 'Gizmoz'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [130]:
filters = {"category_code": "web",  "number_of_employees":{"$gte": 4000}}
projection = {'name': 1, "_id": 0}
results = list(c.find(filters, projection).sort("number_of_employees", 1).limit(10))
results

[{'name': 'Expedia'},
 {'name': 'AOL'},
 {'name': 'Webkinz'},
 {'name': 'Rakuten'},
 {'name': 'Los Angeles Times Media Group'},
 {'name': 'Groupon'},
 {'name': 'Yahoo!'},
 {'name': 'eBay'},
 {'name': 'Experian'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [139]:
filters = {"acquisition.price_amount": {"$gte": 10000000},"acquisition.price_currency_code":"EUR"}
projection = {'name': 1, "_id": 0}
results = list(c.find(filters, projection))
results

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [141]:
filters = {"acquisition.acquired_month": {"$lte": 3}}
projection = {'name': 1, "_id": 0 ,"acquisition": 1 }
results = list(c.find(filters, projection).limit(10))
results

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [145]:
filters = {"founded_year": {"$gte": 2000, '$lte': 2010}, "acquired_year":{'$ne':2011} }
projection = {'name': 1, "_id": 0}
results = list(c.find(filters, projection))
results

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Geni'},
 {'name': 'Twitter'},
 {'name': 'StumbleUpon'},
 {'name': 'Gizmoz'},
 {'name': 'Scribd'},
 {'name': 'Slacker'},
 {'name': 'Helio'},
 {'name': 'MeetMoi'},
 {'name': 'Joost'},
 {'name': 'Babelgum'},
 {'name': 'Plaxo'},
 {'name': 'Powerset'},
 {'name': 'Technorati'},
 {'name': 'AddThis'},
 {'name': 'OpenX'},
 {'name': 'Mahalo'},
 {'name': 'Sparter'},
 {'name': 'Kyte'},
 {'name': 'Veoh'},
 {'name': 'Thoof'},
 {'name': 'Jingle Networks'},
 {'name': 'Meetup'},
 {'name': 'LifeLock'},
 {'name': 'Wesabe'},
 {'name': 'Jangl SMS'},
 {'name': 'SmugMug'},
 {'name': 'Prosper'},
 {'name': 'Jajah'},
 {'name': 'Skype'},
 {'name': 'YouTube'},
 {'name': 'Stickam'},
 {'name': 'blogTV'},
 {'name': 'Livestream'},
 {'name': 'Ustream'},
 {'name': 'AdaptiveBlue'},
 {'name': 'Pando Networks'},
 {'name': 'GrandCentral'},
 {'name': 'Ikan'},
 {'name': 'delicious'},
 {'name': 'Topix'},
 {'n

### 20. All the companies that have been 'deadpooled' after the third year.

In [147]:
filters = {"deadpooled_year": {"$gte": 3}}
projection = {'name': 1, "_id": 0}
results = list(c.find(filters, projection))
results

[{'name': 'Zoho'},
 {'name': 'Omnidrive'},
 {'name': 'Babelgum'},
 {'name': 'Sparter'},
 {'name': 'Thoof'},
 {'name': 'Mercora'},
 {'name': 'Wesabe'},
 {'name': 'Jangl SMS'},
 {'name': 'Stickam'},
 {'name': 'AllPeers'},
 {'name': 'EQO'},
 {'name': 'AllofMP3'},
 {'name': 'SellABand'},
 {'name': 'Zlio'},
 {'name': 'Jaiku'},
 {'name': 'Spotplex'},
 {'name': 'TripUp'},
 {'name': 'Fleck'},
 {'name': 'Coghead'},
 {'name': 'Zooomr'},
 {'name': 'Swivel'},
 {'name': 'Slide'},
 {'name': 'PlaceBlogger'},
 {'name': 'TripHub'},
 {'name': 'Sway'},
 {'name': 'adBrite'},
 {'name': 'Edgeio'},
 {'name': 'coComment'},
 {'name': 'Hotelicopter'},
 {'name': 'Ujogo'},
 {'name': 'PeerMe'},
 {'name': 'CollegeWikis'},
 {'name': 'Hotswap'},
 {'name': 'SplashCast'},
 {'name': 'Tangler'},
 {'name': 'TalkPlus'},
 {'name': 'Citizenbay'},
 {'name': 'PermissionTV'},
 {'name': 'WebbAlert'},
 {'name': 'Nirvanix'},
 {'name': 'Meetro'},
 {'name': 'MyToons'},
 {'name': 'KnockaTV'},
 {'name': 'Millsberry'},
 {'name': 'Revve

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```